[Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course)

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

## Read Datasets

In [ ]:
import pandas as pd

train_data = pd.read_csv('../input/home-data-for-ml-course/train.csv', index_col='Id')
test_data = pd.read_csv('../input/home-data-for-ml-course/test.csv', index_col='Id')

In [ ]:
train_data.head()

In [ ]:
train_data.shape, test_data.shape

## Creating the input and output variables

In [ ]:
# Remove rows with missing target, separate target from predictors
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
X = train_data.drop(['SalePrice'], axis=1)
y = train_data.SalePrice              

## Selecting numeric and categorical columns

In [ ]:
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]
categorical_cols = [cname for cname in X.columns if X[cname].nunique() < 10 and X[cname].dtype == "object"]

# Concating columns
my_cols = categorical_cols + numerical_cols

# Copying datasets
X_data = X[my_cols].copy()
X_test = test_data[my_cols].copy()

## Creating pipeline

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=
                       [('preprocessor', preprocessor),
                        ('model', model)])

## Using cros-validation

In [ ]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("Average MAE score:", scores.mean())

## Creating a function for evaluating scores

In [ ]:
def get_score(n_estimators):
    """Return the average MAE over 3 CV folds of random forest model.
    
    Keyword argument:
    n_estimators -- the number of trees in the forest
    """
    # Replace this body with your own code
    my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators, random_state=0))
    ])
    scores = -1 * cross_val_score(my_pipeline, train_data, y,
                                  cv=3,
                                  scoring='neg_mean_absolute_error')
    return scores.mean()

## Finding best score  

In [ ]:
results = {}
for i in range(3,8):
    results[50*i] = get_score(50*i)

## Plot the scores

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(list(results.keys()), list(results.values()))
plt.show()

## Creating the model and the pipeline

In [ ]:
model = RandomForestRegressor(n_estimators=300, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=
                       [('preprocessor', preprocessor),
                        ('model', model)])

## Refitting model and predicting test set

In [ ]:
my_pipeline.fit(X_data, y)
preds_test = my_pipeline.predict(X_test)

Submitting to competition

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)